In [20]:
import json
import pandas as pd
import numpy as np

## Ribonanza

In [21]:
def rename_families(x):
    if x == '5s' or x == '16s' or x == '23s' or 'rRNA' in x:
        return 'rRNA'
    if x == 'grp1' or x == 'group_I_intron':
        return 'Introns'
    if x == 'grp2' or 'intron' in x:
        return 'Introns'
    if x == 'srp' or x == 'SRP':
        return 'SRP'
    if x == 'telomerase':
        return 'telomerase'
    if x == 'RNaseP':
        return 'rRNA'
    else: 
        return x

## RNAStralign

In [22]:
import rouskinhf
data = rouskinhf.get_dataset('RNAstralign')
families_rnastralign = pd.DataFrame.from_dict(data, orient='index')['family'].apply(lambda x: x.split('__')[0].replace('_database', '')).apply(rename_families)
fam_rnastralign = families_rnastralign.value_counts()
len_rnastralign = pd.DataFrame.from_dict(data, orient='index')['sequence'].apply(len)
families_rnastralign = pd.merge(families_rnastralign,  pd.DataFrame.from_dict(data, orient='index')['sequence'], left_index=True, right_index=True)

## archiveII

In [23]:
data = rouskinhf.get_dataset('archiveII')
families_archivII = pd.DataFrame.from_dict(data, orient='index').reset_index()['index'].apply(lambda x: x.split('_')[0]).apply(rename_families)
fam_archivII = families_archivII.value_counts()
len_archivII = pd.DataFrame.from_dict(data, orient='index')['sequence'].apply(len)
families_archivII = pd.merge(families_archivII,  pd.DataFrame.from_dict(data, orient='index')['sequence'].reset_index(drop=True), left_index=True, right_index=True).rename(columns={'index': 'family'})
families_archivII

,family,sequence
0,rRNA,UGCCUGGCGGCCAUAGUGCGGUGGUCCCACCUGACCCCAUGCCGAA...
1,rRNA,GCCUACGGCCACACCACACUGAAUGCGCCUGAUCUCGUCUGAUCUC...
2,SRP,GCUGGCGGGCCCCUUCGCAUGGUUCGGCGGUGAAUCUGGUCAGGUC...
3,rRNA,UUGCUUGGCGACCAUAGCGUUUUGGACCCACCUGAAUCCAUUCCGA...
4,tmRNA,GGGGGCGUUCUUGGAUUCGACGGGGAUUGCGAGGCUCGAGGUGCAU...
...,...,...
3481,tRNA,GGUUCUCUGGCCGAGUGGUCUAAGGCGCAUGGUUAAGGUCCAUGUC...
3482,rRNA,AGAGGGGUGAGAAUCCCUCCACCGAAUGCCUAAGGGUUCCUGAGGA...
3483,rRNA,UACCUGGUUGAUCCUGCCAGUAGCAUAUGCUUGUCUCAAAGAUUAA...
3484,rRNA,GAUCUGGUGGCCAUGGCGGGGCGCAAUCACCCGAUCCCAUCCCGAA...


## bpRNA (own analysis)


In [24]:
data = rouskinhf.get_dataset('bpRNA')
df = pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'reference'})
df['family'] = df['reference'].apply(lambda x: x.split('_')[1])
df['sequence'] = df['sequence'].apply(lambda x: x.replace('T', 'U'))
df.set_index('reference', inplace=True)
print(len(df))  

66715


In [39]:
# import os
# import numpy as np
# import pandas as pd
# import tqdm
# os.listdir('/Users/yvesmartin/data/bpRNA/')

# def read_bpseq(f, file):
#     ref = file.split('.')[0]
#     lines = f.readlines()
#     seq = ''
#     for line in lines:
#         if line[0] == '#':
#             continue
#         else:
#             if len(line.split()) > 1:
#                 seq += line.split()[1]
#     return {'reference': ref, 'sequence': seq, 'length': len(seq)}

# data = []
# for file in tqdm.tqdm(os.listdir('/Users/yvesmartin/data/bpRNA/'), total=len(os.listdir('/Users/yvesmartin/data/bpRNA/'))):
#     with open('/Users/yvesmartin/data/bpRNA/' + file) as f:
#         data.append(read_bpseq(f, file))

df = pd.read_feather('/Users/yvesmartin/data/bpRNA/bpRNA.feather')
df['family'] = df['reference'].apply(lambda x: x.split('_')[1])
df

,reference,sequence,length,family
0,bpRNA_CRW_9331,CGUGUCGUGAGAUGUUGGGUUAAGUCCCGCAACGAGCGCAACCCUU...,420,CRW
1,bpRNA_CRW_16343,CCUGACGACCAUAGCGAGCGGGUCCCACCUGACCCCAUGCCGAACU...,116,CRW
2,bpRNA_CRW_13525,CUUUGACGUUAGCGGCGGACGGGUGAGUAACACGUGGAUAACCUAC...,989,CRW
3,bpRNA_CRW_14876,GAUGAACGCUGGCGGCGUGCCUAAUACAUGCAAGUCGAGCGAACAG...,1484,CRW
4,bpRNA_CRW_50032,GUCAGGAUAGCUCAGUUGGUAGAGCAGAGGACUGAAAAUC,40,CRW
...,...,...,...,...
102314,bpRNA_CRW_15161,GAUCCUGGCUCAGGACGAACGCUGGCGGCGUGCCUAAUACAUGCAA...,1537,CRW
102315,bpRNA_CRW_22308,GGGGGCUUAGCUCAGCUGGGAGAGCGCCUGCUUUGCACGCAGGAGG...,73,CRW
102316,bpRNA_RFAM_19577,UGUGCAUCGUGGUCAAAUGCUCAGACUCCUGUGGUGGCUGCUCAUG...,81,RFAM
102317,bpRNA_RFAM_23681,CCCAAAGGUUCCCUCAGGCUGAAUGGAAACCAGCCAGAGAGUGUAA...,108,RFAM


### bpRNA/labeled

In [40]:
valid_labels = list(fam_archivII.keys()) + list(fam_rnastralign.keys()) + ['SPR', 'RNP', 'ncRNA']
df['family'] = df['family'].apply(lambda x: 'ncRNA' if x == 'PDB' else x)
df_labeled = df[df['family'].isin(valid_labels)].copy()
df_labeled['family'].value_counts()

family
SRP      959
tmRNA    728
ncRNA    669
SPR      623
RNP      466
Name: count, dtype: int64

### bpRNA/unlabeled

In [41]:
df_un = df[~df['family'].isin(valid_labels)].copy()
df_un = df_un[['sequence']].reset_index().rename(columns={'index': 'reference'})
len(df_un)

98874

### Add labels from external databases through sequence matching

#### Rfam fasta files

In [42]:
rfam = pd.read_csv('/Users/yvesmartin/src/supermodels-data/rfam/data/rfam.csv')
rfam = rfam[['ref_desc', 'sequence', 'family_name', 'full_family_name', 'clan_name']]
rfam['sequence'] = rfam['sequence'].apply(lambda x: x.replace('T', 'U'))

In [44]:
df = pd.merge(df_un, rfam, on='sequence', how='left')
df.sort_values(by='family_name', inplace=True, na_position='last')
df.drop_duplicates(subset=['reference'], inplace=True, keep='first')
df

,reference,sequence,ref_desc,family_name,full_family_name,clan_name
826485,69039,CCUUCAUUGGUUUACCUCAAACCUGUUGUGAUGUAAGUUAAUGAAG...,"Streptococcus sanguinis SK36, complete genome.",23S-methyl,23S methyl RNA motif,NaN
792602,66386,CAUUCGUUGGUUUAAAUCAAACCUGUUAUGAUUUAAGUUAGCGAGU...,"Streptococcus pyogenes M1 GAS, complete genome.",23S-methyl,23S methyl RNA motif,NaN
50361,4686,UUUUCAUUGGUUUUUAUCAGGUUCCUGUUCUGAUAAAAGUUAGUGA...,Enterococcus durans ATCC 6056 genomic scaffold...,23S-methyl,23S methyl RNA motif,NaN
342385,28544,GAUUUGUUAGUUUAAAUCAAACCUGUUAUGAUUUAAGUUAACAAAC...,"Streptococcus agalactiae 2603V/R, complete gen...",23S-methyl,23S methyl RNA motif,NaN
463875,39125,GAUUUGUUAGUUUAAAUCAAACCUGUUAUGAUUUAAGCUAACAAAC...,Streptococcus agalactiae H36B s_agalactiae_h36...,23S-methyl,23S methyl RNA motif,NaN
...,...,...,...,...,...,...
1211827,102312,GAAUUCCACGUGUAGCGGUGAAAUGCGUAGAGAUGUGGAGGAACAC...,NaN,NaN,NaN,NaN
1211828,102313,GGGCUUGUAGCUCAGUUGGUUAGAGCGCGCGCUUGAUAAGCGUGAG...,NaN,NaN,NaN,NaN
1211829,102314,GAUCCUGGCUCAGGACGAACGCUGGCGGCGUGCCUAAUACAUGCAA...,NaN,NaN,NaN,NaN
1211885,102317,CCCAAAGGUUCCCUCAGGCUGAAUGGAAACCAGCCAGAGAGUGUAA...,NaN,NaN,NaN,NaN


#### CRW 

In [45]:
data = pd.DataFrame.from_dict(json.load(open('/Users/yvesmartin/src/supermodels-data/CRW/crw.json')), orient='index').reset_index().rename(columns={'index': 'reference', 'family':'family_name'})[['sequence', 'family_name']]
df = pd.merge(df, data, on='sequence', how='left')
df['family_name'] = df.apply(lambda x: x['family_name_x'] if pd.isnull(x['family_name_y']) else x['family_name_y'], axis=1)
df.drop(['family_name_x', 'family_name_y'], axis=1, inplace=True)
df['family_name'].value_counts()

family_name
tRNA       9785
5S_rRNA    1040
5S          927
16S         709
tmRNA       477
           ... 
mir-885       1
mir-995       1
mir-988       1
mir-965       1
mir-136       1
Name: count, Length: 1604, dtype: int64

In [46]:
translation_family_name = {
    'sRNA': ['sRNA', 'rli', 'rivX', 'CC', 'STnc', 'sau-', 'Atu_', 'GlsR', '6C', 'ArcZ', 'Bsr', 'C0', 'CyaR_RyeE', 'whalefall-1', 'tpke11', 'tfoR',\
        't44', 'sro', 'FsrA', 'sraA', 'ryfA', 'GadY', 'GcvB', 'Hgc', 'IS009', 'InvR', 'MtlS', 'OmrA-B', 'OrzO-P', 'OxyS', 'PrrF', 'Qrr', 'RybB', 'RydC', 'SgrS',\
        'Spot_42', 'Sra'
            ],
    'tRNA': ['tRNA', 'TLS-PK'],
    'rRNA': ['rRNA', '5s', '5S_rRNA', '16s', '16S_rRNA', '23s', '23S_rRNA', 'RF_site', 'ribozyme', 'GOLLD'],
    'ncRNA': ['ncRNA', 'rdlD', 'NRON', 'CopA', 'DicF', 'uc_338', 'Dicty_Class_I_RNA', 'DsrA', 'FourU', 'srg1', 'sok', 'symR', 'sar', 'rncO', \
            'rydB', 'msr', 'IS102', 'IS128', 'MicC', 'MicF', 'NrrF', 'NsiR1', 'Plasmid_RNAIII','RNA-OUT', 'RNAI','RUF', \
             'RprA', 'Rsa', 'SprD'
                ],
    'group_I_intron': ['grp1'],
    'group_II_intron': ['grp2', 'group-II'],
    'SRP': ['srp', 'SRP'],
    'RNaseP': ['RNaseP'],
    'crRNA': ['CRISPR'],
    'RNP': ['HACA'],
    'snRNA': ['snRNA', 'sn', 'Gl_U', 'SNORA', 'U1', 'U7', 'U3', 'VA', 'SCARNA'],
    'microRNA': ['mir-', 'MIR', 'lsy-6'],
    'virus': ['virus', 'CuYV_BPYV', 'SPCSV', 'HAV', 'BMV3_UPD', 'Rubella_3', 'HIV', 'HBV', 'IRES'],
    'tmRNA': ['tmRNA'],
    'mRNA': ['SAM', 'mini-ykkC', 'FIE3'],
    'CRE': ['sucA', 'Antizyme_FSE', 'CAESAR', 'ylbH', 'yjdF', 'ydaO-yuaA', 'ybhL', 'wcaG', 'G-CSF_SLDE', 'GABA3', 'GAIT', 'speF', 'GP_knot', 
            'Gurken', 'K_chan_RES', 'Mg_sensor', 'PyrR', 'RtT', 'SECIS_'                                   ],
    'other': ['IMES-3', 'isrK', 'isrL', 'DapZ', 'ppoRNA', 'DNA', 'RyhB'],
    'telomerase': ['tp2'],
    'motif': ['MS2', 'OLE', 'PYLIS_'],
}

translation_clan_name = {
    'snRNA': ['7SK'],
    'ncRNA': ['Csr_Rsm_clan', 'FinP-traJ','Glm', 'suhB'],
    'sRNA': ['RyeA-RyeB', 'LhrC'],
    'RNaseP': ['RNaseP'],
    'mRNA': ['SL'],
}

translation_family_description = {
    'mRNA': ['riboswitch', 'UTR', 'mRNA', 'promoter', ],
    'snRNA': ['spliceosomal', 'Small nucleolar', 'snoRNA', 'Small Nucleolar RNA', 'small nucleolar'],
    'sRNA': ['sRNA', 'small RNA', 'AniS', 'Anti-Q RNA', 'Hfq binding', 'antisense','anti-sense', 'Antisense', 'anti-toxin', 'Short', 'antitoxin'],
    'virus': ['virus'],
    'microRNA': ['microRNA'],
    'tRNA': ['tRNA'],
    'tmRNA': ['tmRNA'],
    'crRNA': ['CRISPR'],
    'telomerase': ['telomerase'],
    '23S_rRNA': ['23S'],
    '6S_rRNA': ['6S'],
    'motif': ['pseudoknot', 'Hammerhead', 'motif', 'Stem loopII regulatory element in POLB', 'Pseudoknot'],
    'CRE': ['Alpha operon ribosome binding site', 'leader', 'cis-regulatory element', 'element'],
    'rRNA': ['rRNA', 'Ribosomal', 'ribosomal', 'ribozyme', 'pRNA', 'ribosome'],
    'ncRNA': ['Y RNA', 'thermometer', 'noncoding', 'SscA', 'Non-coding'], 
    'group_I_intron': ['Group I'],
}

def clean_family(row):
    x = row['clan_name']
    if not pd.isna(x):
        for family, v in translation_clan_name.items():
            for vv in v:
                if vv in x:
                    return family

    x = row['family_name']
    if pd.isna(x):
        return 'other'
    for family, v in translation_family_name.items():
        for vv in v:
            if vv in x:
                return family
            
    full_family_name = row['full_family_name']
    if pd.isna(full_family_name):
        return 'other'
    for family, keywords in translation_family_description.items():
        for kw in keywords:
            if kw in full_family_name:
                return family     
    return 'other'

def rename_families_bp2(x):
    if x == '5s' or x == '16s' or x == '23s' or 'rRNA' in x:
        return 'rRNA'
    if x == 'grp1' or x == 'group_I_intron' or x == 'Introns':
        return 'Introns'
    if x == 'grp2' or 'intron' in x:
        return 'Introns'
    if x == 'srp' or x == 'SRP':
        return 'Other'
    if x == 'telomerase':
        return 'telomerase'
    if x == 'RNaseP':
        return 'rRNA'
    if x == 'snRNA' or x == 'microRNA' or x == 'CRE' or x == 'sRNA':
        return 'sRNA'
    if x in ['virus', 'motif', 'other']:
        return 'Other'
    else: 
        return x

df_temp = df.copy()
df_temp['family'] = df.apply(clean_family, axis=1)
df_temp['length'] = df_temp['sequence'].apply(len)
df_temp.set_index('reference', inplace=True)
df_final = pd.concat([df_labeled, df_temp])[['sequence', 'family']]
df_final['family'] = df_final['family'].apply(rename_families_bp2)
families_bpRNA = df_final.copy()
bprna = df_final.value_counts('family')
len_bprna = df_final['sequence'].apply(len).tolist()
df_bprna = df_final.copy()

In [32]:
# bprna = pd.DataFrame({
#     'tRNA': 35622,
#     "rRNA": 17641 + 477 + 1425 + 191 + 186,
#     'SRP': 1603,
#     'Introns': 237+131,
#     'tmRNA': 161,
# }, index=[0])

In [47]:
# label bprna 90
df90 =  open('/Users/yvesmartin/data/bpRNA/bpRNA_1m_90.fasta').read().split('\n')


refs, seqs = df90[::2], df90[1::2]
df90 = pd.DataFrame({'reference': [r.split('>')[1] for r in refs if '>' in r], 'sequence': seqs})
df90 = pd.merge(df_final, df90, on='sequence', how='inner')
bprna90 = df90.value_counts('family')
len_bprna90 = df90['sequence'].apply(len).tolist()

In [48]:
# add ribonanza
import rouskinhf
df_ribo = pd.DataFrame.from_dict(rouskinhf.get_dataset('ribo500-blast'), orient='index').reset_index().rename(columns={'index': 'reference'})
df_ribo['database'] = 'Ribonanza'
df_ribo['family'] = 'Ribonanza'

In [56]:
df_bprna['database'] = 'bpRNA'
families_rnastralign['database'] = 'RNAStralign'
families_archivII['database'] = 'archiveII'
df_final = pd.concat([df_bprna, families_rnastralign, families_archivII, df_ribo])
df_final['family'] = df_final['family'].apply(lambda x: np.nan if x == 'Other' else x)
df_final.sort_values(by='family', inplace=True, na_position='last')
df_display = df_final.copy()    
df_display.drop_duplicates(subset=['sequence'], inplace=True, keep='first')
df_display['family'].fillna('Other', inplace=True)
df_display.family.value_counts()

for family_name in df_display.family.unique():
    df_display.loc[df_display['family'] == family_name, 'family'] = family_name + ' (N={:,})'.format(df_display[df_display['family'] == family_name].shape[0])

# display this on a pie chart
import plotly.express as px
import plotly.graph_objects as go

print("final df has {} sequences".format(len(df_display)))

fig = go.Figure(data=[go.Pie(labels=df_display.family.value_counts().index, values=df_display.family.value_counts().values, hole=.3,
                             # show real numbers in textinfo
                                textinfo='label+percent',
                                textposition='inside',
                             )])
fig.update_layout( height=700, width=800)
fig.show()

# to pdf
import plotly.io as pio
pio.write_image(fig, 'images/family_classification.pdf')

final df has 130660 sequences


# Aggregate

In [57]:

df = pd.concat([bprna.T, fam_rnastralign, fam_archivII], axis=1)
df.columns = ['bpRNA-1m', 'RNAStralign', 'ArchiveII']
# df = df.fillna(0).astype(int)
df_family = df.copy()
df_family.loc['total'] = {
    'bpRNA-1m': len(len_bprna),
    # 'bpRNA-1m(90)': len(len_bprna90),
    'RNAStralign': len(len_rnastralign),
    'ArchiveII': len(len_archivII),
}
# others = total - df_family.sum()

# assert df_family.loc[df_family.index[:-1]].sum().sum() == df_family.loc['total'].sum()
df_family.drop('total', inplace=True)
# df_family = df_family.fillna(0).astype(int)
df_family.index.to_list()

['Other',
 'sRNA',
 'tRNA',
 'rRNA',
 'mRNA',
 'ncRNA',
 'Introns',
 'tmRNA',
 'SPR',
 'RNP',
 'telomerase',
 'SRP']

In [58]:
df_family.loc[df_family.index[:-1]].sum()

bpRNA-1m       102983.0
RNAStralign     26637.0
ArchiveII        2761.0
dtype: float64

In [59]:
# one value per bin
def make_histograms(l):
    min_val = 0
    max_val = 4400
    bin_size = 100
    bins = np.arange(min_val + bin_size/2, max_val - bin_size/2, bin_size)
    hist, bins = np.histogram(l, bins=max_val//bin_size, range=(min_val, max_val))
    return hist, bins
hists = {
    'bpRNA': make_histograms(len_bprna),
    'RNAStralign': make_histograms(len_rnastralign),
    'archiveII': make_histograms(len_archivII),
}

### Plot as piecharts

In [60]:
horizontal_spacing = 0.08
vertical_spacing = 0.2
height = 650
width = 1200

In [75]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_final['length'] = df_final['sequence'].apply(len) 
df_final['family'] = df_final['family'].fillna('Other').apply(rename_families_bp2)   

# sort families by count
for family in df_final.family.unique():
    df_final.loc[df_final['family'] == family, 'family_count'] = df_final[df_final['family'] == family].shape[0]
    
for family_name in df_final.family.unique():
    df_final.loc[df_final['family'] == family_name, 'family'] = family_name.split(' (')[0] + ' (N={:,})'.format(df_final[df_final['family'] == family_name].shape[0])
    
df_final.sort_values(by='family_count', ascending=False, inplace=True)

df_family = df_final.groupby('database').apply(lambda x: x['family'].value_counts()).unstack().fillna(0).astype(int).T

df_family['total'] = df_family.sum(axis=1)
df_family.sort_values(by='total', ascending=False, inplace=True)
df_family.drop(columns=['total', 'Ribonanza'], inplace=True)

df_family = df_family[['bpRNA', 'RNAStralign', 'archiveII']]
df_family = df_family.loc[df_family.index != 'Ribonanza (N=46,060)']

df_family

database,bpRNA,RNAStralign,archiveII
family,,,
"Other (N=66,086)",64916,445,725
"rRNA (N=24,206)",3456,19027,1723
"tRNA (N=16,962)",10033,6436,493
"sRNA (N=16,143)",16143,0,0
"mRNA (N=2,688)",2688,0,0
"Introns (N=2,379)",1538,737,104
"tmRNA (N=2,165)",1361,400,404
"ncRNA (N=1,602)",1602,0,0
SPR (N=623),623,0,0


In [76]:


# show length distribution for each family
import plotly.express as px
import plotly.graph_objects as go

# first row is piechart of family distribution
# second row is histogram of sequence length distribution
# I want it to look like a paper figure

if 'bpRNA-1m(90)' in hists.keys():
    del hists['bpRNA-1m(90)'] 

fig = make_subplots(rows=2, cols=3, specs=[[{'type': 'domain'}]*3, [{'type': 'histogram'}]*3], 
                    subplot_titles=[f"{name} (N={np.nansum(df_family[name]).astype(int)})" for name in df_family.columns] + ['']*3,
                    vertical_spacing=vertical_spacing,
                    horizontal_spacing=horizontal_spacing,
                    row_heights=[0.5, 0.2],
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=22)
    

for i, name in enumerate(df_family.columns):
    fig.add_trace(go.Pie(
        labels=df_family.index,
        values=df_family[name],
        name=name,
        textinfo=f'percent',
        texttemplate='%{percent}',
        textposition='inside',
        # colors=px.colors.qualitative.Plotly + [px.colors.qualitative.Plotly[-1]]*10000,
        showlegend=i==0 and name !='Ribonanza',
        # colors
        marker_colors=px.colors.qualitative.Plotly + [px.colors.qualitative.Plotly[-1]]*10000,
        # sort=True,
        # legennd location
        # domain={'x': [0.0, 0.25], 'y': [0.5, 1.0]} if i == 0 else {'x': [0.25, 0.5], 'y': [0.5, 1.0]} if i == 1 else {'x': [0.5, 0.75], 'y': [0.5, 1.0]} if i == 2 else {'x': [0.75, 1.0], 'y': [0.5, 1.0]},
    ), row=1, col=i+1)
    fig.add_trace(go.Bar(
        x=hists[name][1][:-1],
        y=hists[name][0],
        name=name,
        showlegend=False,
        marker_color='rgb(0, 0, 0)',
    ), row=2, col=i+1)
    # fig.update_yaxes(row=2, col=i+1,  range=[0, 50000])
    fig.update_xaxes(row=2, col=i+1, range=[-100, 2000], tick0=0, dtick=2000)
    # make bar width constant
    # add horitontal lines
    fig.update_layout(barmode='overlay', bargap=0.1, bargroupgap=0.1)

# write 'sequence length' at the bottom, centered
fig.add_annotation(dict(
    x=0.5,
    y=-0.15,
    text='Sequence length (bin size: 100)',
    showarrow=False,
    font=dict(size=22),
    xref='paper',
    yref='paper',
    xanchor='center',
    yanchor='bottom',
))

fig.add_annotation(dict(
    x=-0.06,
    y=0.03,
    text='Count',
    showarrow=False,
    font=dict(size=22),
    xref='paper',
    yref='paper',
    xanchor='center',
    yanchor='bottom',
    textangle=-90
), 
)

fig.update_layout(
    height=height,
    width=width,
    title_x=0.5,
    title_y=0.95,
    font_size=20,
    font_family='Arial',
    font_color='black',
    legend_font_size=20,
    legend_font_family='Arial',
    legend_font_color='black',
    legend_x=1.03,
    legend_y=0.95,
    # legend_orientation='h',
    legend_traceorder='normal',
    legend_bordercolor='black',
    template="plotly_white",
)

fig.show()

# save to pdf
import plotly.io as pio
pio.write_image(fig, 'images/family_available_data.pdf')


# Length distribution

In [145]:

class Color:
    def __init__(self):
        self.colors = px.colors.qualitative.Plotly
        self.i = 0
    
    def __iter__(self):
        for color in self.colors:
            yield color
        yield self.colors[-1]
        
    def __len__(self):
        return len(self.colors) + 10000
    
    
px.colors.qualitative.Plotly

['#636EFA',
 '#EF553B',
 '#00CC96',
 '#AB63FA',
 '#FFA15A',
 '#19D3F3',
 '#FF6692',
 '#B6E880',
 '#FF97FF',
 '#FECB52']

In [152]:


fig = px.histogram(df_final, x="length", color="family",nbins=int(np.ceil(df_final['length'].max()/100)), range_x=[0, 2000],
                   color_discrete_sequence= px.colors.qualitative.Plotly + [px.colors.qualitative.Plotly[-1]]*10000,
)

fig.update_layout(
    height=700,
    width=1200,
    title_x=0.5,
    title_y=0.95,
    font_size=20,
    font_color='black',
    legend_font_size=20,
    legend_font_family='Arial',
    legend_font_color='black',
    legend_x=1.03,
    legend_y=0.95,
    legend_traceorder='normal',
    legend_bordercolor='black',
    template="plotly_white",
    bargap=0.2,
)

# write the number of values above 2000 
fig.add_annotation(dict(
    x=0.95,
    y=0.93,
    text='Count of sequences above 2,000 nt: {}'.format(len(df_final[df_final['length'] > 2000])),
    showarrow=False,
    font=dict(size=22),
    xref='paper',
    yref='paper',
    xanchor='right',
    yanchor='top',
))

# add frame around the plot
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
# make log
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig.show()

# save to pdf
import plotly.io as pio
pio.write_image(fig, 'images/family_length_distribution.pdf')